In [19]:
# user-configurable
source = 'nj'
route = '119'
stop = '30189'
db_server='192.168.1.181'
# db_server='127.0.0.1'


In [20]:
# import libraries
import datetime, time, sys
from operator import itemgetter
import pandas as pd

In [21]:
# import NJT API app libraries
import reportcard.lib.BusAPI as BusAPI

In [22]:
# work with bus position log
import reportcard.lib.BusRouteLogsDB as BusRouteLogsDB
db = BusRouteLogsDB.MySQL('buses', 'buswatcher', 'njtransit', db_server, route)
conn = db.conn
table_name = 'routelog_' + route

something went wrong with mysql


SystemExit: 

In [12]:
# just get a specific date out
query = ('SELECT * FROM routelog_119 WHERE (rt=119 AND datediff(timestamp, "20180927") = 0 ) ORDER BY run, timestamp;')

In [13]:
# get a single days positions table into a dataframe
df = pd.read_sql_query(query, conn)
df

,pkey,lat,lon,bid,cars,consist,d,dip,dn,fs,...,pdRtpiFeedName,pid,rt,rtRtpiFeedName,rtdd,rtpiFeedName,run,wid1,wid2,timestamp
0,24526,40.757130,-73.992447,8136,,,West Bound,1678,WSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 14:46:01.407114
1,24534,40.757130,-73.992447,8136,,,West Bound,1678,WSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 14:47:00.946658
2,24542,40.757130,-73.992447,8136,,,West Bound,1678,WSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 14:48:04.154599
3,24549,40.757130,-73.992447,8136,,,West Bound,1678,WSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 14:49:05.147091
4,24556,40.757130,-73.992447,8136,,,West Bound,1678,WSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 14:50:08.889733
5,24563,40.757481,-73.993149,8136,,,North Bound,1919,NNE,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 14:51:01.877115
6,24570,40.757259,-73.992767,8136,,,North West Bound,1919,NW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 14:52:02.492105
7,24578,40.757820,-73.993668,8136,,,North West Bound,2117,WNW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 14:53:01.569098
8,24586,40.757969,-73.996552,8136,,,North West Bound,2969,NW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 14:54:02.488277
9,24594,40.759171,-73.999977,8136,,,North West Bound,4019,WNW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 14:55:02.697860


In [14]:
# split final approach history (sorted by timestamp) at each change in vehicle_id outputs a list of dfs -- per https://stackoverflow.com/questions/41144231/python-how-to-split-pandas-dataframe-into-subsets-based-on-the-value-in-the-fir

run_dfs = [g for i, g in df.groupby(df['run'].ne(df['run'].shift()).cumsum())]
run_dfs

[     pkey        lat        lon   bid cars consist                 d    dip  \
 0   24526  40.757130 -73.992447  8136                     West Bound   1678   
 1   24534  40.757130 -73.992447  8136                     West Bound   1678   
 2   24542  40.757130 -73.992447  8136                     West Bound   1678   
 3   24549  40.757130 -73.992447  8136                     West Bound   1678   
 4   24556  40.757130 -73.992447  8136                     West Bound   1678   
 5   24563  40.757481 -73.993149  8136                    North Bound   1919   
 6   24570  40.757259 -73.992767  8136               North West Bound   1919   
 7   24578  40.757820 -73.993668  8136               North West Bound   2117   
 8   24586  40.757969 -73.996552  8136               North West Bound   2969   
 9   24594  40.759171 -73.999977  8136               North West Bound   4019   
 10  24601  40.760448 -74.003166  8136               North West Bound   5017   
 11  24608  40.760448 -74.003166  8136  

In [15]:
len(run_dfs)

23

In [16]:
results = pd.DataFrame()

for run in run_dfs:
    start_report = run.head(1)
    end_report = run.tail(1)
    results = results.append(run.head(1))
    results = results.append(run.tail(1))
    

In [17]:
results

,pkey,lat,lon,bid,cars,consist,d,dip,dn,fs,...,pdRtpiFeedName,pid,rt,rtRtpiFeedName,rtdd,rtpiFeedName,run,wid1,wid2,timestamp
0,24526,40.757130,-73.992447,8136,,,West Bound,1678,WSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 14:46:01.407114
70,25318,40.652791,-74.135161,8136,,,South Bound,70159,S,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-27 19:44:00.840094
71,23535,40.759020,-74.028062,8101,,,North Bound,17365,N,119 NEW YORK,...,,99396,119,,119,,12,012,0010,2018-09-27 09:27:33.662040
165,25372,40.677645,-74.114655,8274,,,South Bound,59237,SSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,12,012,0119,2018-09-27 19:49:01.353794
166,25084,40.757865,-73.992995,8118,,,South East Bound,2497,ESE,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,13,013,0119,2018-09-27 19:24:01.222124
185,25365,40.748863,-74.040784,8118,,,North West Bound,21032,WNW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,13,013,0119,2018-09-27 19:49:01.353794
186,24629,40.721927,-74.075905,8116,,,South Bound,39458,SSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,205,205,0119,2018-09-27 18:41:00.945056
214,24981,40.647854,-74.138349,8116,,,South West Bound,72190,SSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,205,205,0119,2018-09-27 19:14:01.214357
215,24632,40.685369,-74.109044,8276,,,South West Bound,55990,SSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,301,301,0010,2018-09-27 18:41:00.945056
224,24748,40.650182,-74.136462,8276,,,South West Bound,71195,SSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,301,301,0010,2018-09-27 18:55:01.840288


In [18]:
results.to_csv('119_run_report.csv')